# Static Mode Analysis: Complete Workflow

This notebook covers the complete static-mode analysis workflow:

1. Loading and validating experimental data
2. Parameter exploration and initial value selection
3. NLSQ fitting with diagnostics
4. Multi-start optimization for robustness
5. Result visualization and physical interpretation

**Use static mode when:** your sample is in equilibrium (no shear/flow) and
you see no angular dependence in C2.

---

## 1. Setup and Data Loading

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from homodyne.config import ConfigManager
from homodyne.data import validate_xpcs_data
from homodyne.optimization.nlsq import fit_nlsq_jax
from homodyne.utils.logging import get_logger, log_phase

logger = get_logger(__name__)

In [ ]:
# Generate synthetic static-mode data
# (Replace with: data = load_xpcs_data("your_config.yaml"))

rng = np.random.default_rng(seed=123)

# True parameters for validation
TRUE = {
    "D0": 850.0,  # Å²/s
    "alpha": -0.35,  # Mild sub-diffusion
    "D_offset": 0.08,  # Å²/s
    "beta_contrast": 0.15,
    "offset": 1.0,
}

q = 0.054  # Å⁻¹
n_t = 50
n_phi = 8
dt = 0.1
t = dt * np.arange(n_t)
phi_angles = np.linspace(0, 315, n_phi)  # 0° to 315° in steps of 45°

# Precompute D(t) on the time grid and its cumulative trapezoid
D_t = TRUE["D0"] * t ** TRUE["alpha"] + TRUE["D_offset"]
trap_avg = 0.5 * (D_t[:-1] + D_t[1:])
D_cumsum = np.concatenate([[0.0], np.cumsum(trap_avg)])

c2 = np.zeros((n_phi, n_t, n_t))
for i_phi in range(n_phi):
    for i_t1 in range(n_t):
        for i_t2 in range(i_t1, n_t):
            # Diffusion integral (cumulative trapezoid)
            D_integral = abs(D_cumsum[i_t2] - D_cumsum[i_t1]) * dt
            g1_sq = np.exp(-2 * q**2 * D_integral)
            val = TRUE["offset"] + TRUE["beta_contrast"] * g1_sq
            noise = 0.003 * rng.standard_normal()
            c2[i_phi, i_t1, i_t2] = val + noise
            c2[i_phi, i_t2, i_t1] = val + noise  # symmetric

data = {
    "c2_exp": c2,
    "t1": t,
    "t2": t,
    "phi_angles_list": phi_angles,
    "wavevector_q_list": np.array([q]),
    "sigma": 0.003 * np.ones_like(c2),
    "L": 5.0e6,
    "dt": dt,
}

print(f"Data shape: {c2.shape}  # (n_phi={n_phi}, n_t1={n_t}, n_t2={n_t})")
print(f"q = {q} Å⁻¹,  {n_phi} angles,  dt = {dt} s")

## 2. Data Validation

In [ ]:
# Validate the loaded data
report = validate_xpcs_data(data)

print(f"Data valid: {report.is_valid}")
if report.warnings:
    print(f"Warnings:   {[w.message for w in report.warnings]}")
if report.errors:
    print(f"Errors:     {[e.message for e in report.errors]}")

# Check angular dependence (should be minimal for static mode)
lag_idx = 3
c2_at_lag = np.array([c2[i_phi, 0, lag_idx] for i_phi in range(n_phi)])
angular_var_ratio = c2_at_lag.std() / c2_at_lag.mean()
print(f"\nAngular variance ratio at lag={lag_idx}: {angular_var_ratio:.4f}")
if angular_var_ratio < 0.02:
    print("→ No significant angular dependence → static mode is appropriate")
else:
    print("→ Angular dependence detected → consider laminar_flow mode")

## 3. Parameter Space Exploration

Before fitting, it's useful to understand the parameter landscape.

In [ ]:
# Explore how D0 affects the decorrelation curve
def model_c2_1d(t_arr, D0, alpha, D_offset, contrast, offset, q):
    """Simple 1D C2 model (time-averaged, no angle dependence)."""
    dt_grid = t_arr[1] - t_arr[0] if len(t_arr) > 1 else 1.0
    # Precompute D(t) on the time grid and its cumulative trapezoid
    D_t = D0 * t_arr**alpha + D_offset
    trap_avg = 0.5 * (D_t[:-1] + D_t[1:])
    D_cumsum = np.concatenate([[0.0], np.cumsum(trap_avg)])
    c2_vals = []
    for i_t2 in range(len(t_arr)):
        # Diffusion integral (cumulative trapezoid)
        D_integral = abs(D_cumsum[i_t2] - D_cumsum[0]) * dt_grid
        g1_sq = np.exp(-2 * q**2 * D_integral)
        c2_vals.append(offset + contrast * g1_sq)
    return np.array(c2_vals)


fig, axes = plt.subplots(1, 3, figsize=(13, 4))

# Vary D0
ax = axes[0]
for D0_test in [200, 500, 850, 2000, 5000]:
    c2_model = model_c2_1d(
        t,
        D0_test,
        TRUE["alpha"],
        TRUE["D_offset"],
        TRUE["beta_contrast"],
        TRUE["offset"],
        q,
    )
    ax.semilogx(t, c2_model, label=f"D0={D0_test}")
ax.set_xlabel("t (s)")
ax.set_ylabel("C2")
ax.set_title("Effect of D0")
ax.legend(fontsize=8)

# Vary alpha
ax = axes[1]
for alpha_test in [-1.5, -1.0, -0.5, 0.0, 0.5]:
    c2_model = model_c2_1d(
        t,
        TRUE["D0"],
        alpha_test,
        TRUE["D_offset"],
        TRUE["beta_contrast"],
        TRUE["offset"],
        q,
    )
    ax.semilogx(t, c2_model, label=f"α={alpha_test}")
ax.set_xlabel("t (s)")
ax.set_title("Effect of alpha")
ax.legend(fontsize=8)

# Vary contrast
ax = axes[2]
for beta_test in [0.05, 0.10, 0.15, 0.25, 0.40]:
    c2_model = model_c2_1d(
        t, TRUE["D0"], TRUE["alpha"], TRUE["D_offset"], beta_test, TRUE["offset"], q
    )
    ax.semilogx(t, c2_model, label=f"β={beta_test}")
ax.set_xlabel("t (s)")
ax.set_title("Effect of contrast (β)")
ax.legend(fontsize=8)

plt.suptitle("Parameter Sensitivity Analysis", fontsize=12)
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Single-Start NLSQ Fit

In [ ]:
import tempfile

config_yaml = """
analysis_mode: "static"

analyzer_parameters:
  dt: 0.1

optimization:
  method: "nlsq"
  nlsq:
    anti_degeneracy:
      per_angle_mode: "auto"

initial_parameters:
  parameter_names: [D0, alpha, D_offset]
  values: [500.0, -0.5, 0.1]
"""

with tempfile.NamedTemporaryFile(mode="w", suffix=".yaml", delete=False) as f:
    f.write(config_yaml)
    config_path = f.name

config = ConfigManager(config_path)

with log_phase("NLSQ Single-Start"):
    result_single = fit_nlsq_jax(data, config)

# Physical parameters come after per-angle scaling
phys_offset = 2 * n_phi

print(f"Status:      {result_single.convergence_status}")
print(f"chi^2_nu:    {result_single.reduced_chi_squared:.4f}")
print(f"Time:        {result_single.execution_time:.2f} s")
print()
print(f"{'Param':<12} {'True':>10} {'Fitted':>10} {'Err':>10}")
print("-" * 46)
for i, name in enumerate(["D0", "alpha", "D_offset"]):
    tv = TRUE[name]
    fv = result_single.parameters[phys_offset + i]
    fe = result_single.uncertainties[phys_offset + i]
    print(f"{name:<12} {tv:>10.4g} {fv:>10.4g} {fe:>10.4g}")

## 5. Multi-Start Optimization

For robust results, run multiple starts and take the best. Multi-start
explores the parameter space using Latin Hypercube Sampling to avoid
local minima.

In [ ]:
# Multi-start with Latin Hypercube Sampling
# Multi-start is configured via the YAML config
from homodyne.optimization.nlsq import fit_nlsq_multistart

ms_config_yaml = """
analysis_mode: "static"

analyzer_parameters:
  dt: 0.1

optimization:
  method: "nlsq"
  nlsq:
    anti_degeneracy:
      per_angle_mode: "auto"
    multi_start:
      enable: true
      n_starts: 10
      sampling_strategy: "latin_hypercube"

initial_parameters:
  parameter_names: [D0, alpha, D_offset]
  values: [500.0, -0.5, 0.1]
"""

with tempfile.NamedTemporaryFile(mode="w", suffix=".yaml", delete=False) as f:
    f.write(ms_config_yaml)
    ms_config_path = f.name

config_ms = ConfigManager(ms_config_path)

with log_phase("NLSQ Multi-Start"):
    result_multi = fit_nlsq_multistart(data, config_ms)

print(f"Best chi^2_nu:        {result_multi.best.reduced_chi_squared:.4f}")
print(
    f"Starts converged:     {result_multi.n_successful}/{len(result_multi.all_results)}"
)
print(f"Unique basins:        {result_multi.n_unique_basins}")
print()

# Show spread of D0 estimates across starts to assess uniqueness
D0_estimates = [
    s.final_params[phys_offset] for s in result_multi.all_results if s.success
]
if D0_estimates:
    print(
        f"D0 across converged starts: mean={np.mean(D0_estimates):.1f},"
        f" std={np.std(D0_estimates):.1f} Å²/s"
    )
    print("(Small std → unique global minimum; large std → multi-modal)")

import os

os.unlink(ms_config_path)

## 6. Comparing Starting Points

In [ ]:
# Test sensitivity to initial parameters
# Note: initial_params must include contrast and offset alongside physical parameters
initial_conditions = [
    {"D0": 100, "alpha": -1.0, "D_offset": 0.01, "label": "Far below"},
    {"D0": 500, "alpha": -0.5, "D_offset": 0.1, "label": "Near true"},
    {"D0": 5000, "alpha": 0.5, "D_offset": 1.0, "label": "Far above"},
    {"D0": 850, "alpha": -0.3, "D_offset": 0.05, "label": "Very close"},
]

results_ic = []
for ic in initial_conditions:
    init = {
        "contrast": 0.15,  # Estimated speckle contrast
        "offset": 1.0,  # Baseline offset
        "D0": ic["D0"],
        "alpha": ic["alpha"],
        "D_offset": ic["D_offset"],
    }
    r = fit_nlsq_jax(data, config, initial_params=init)
    results_ic.append({"label": ic["label"], "result": r})

print(
    f"{'Starting point':<18} {'D0 fitted':>12} {'alpha':>10} {'chi2_nu':>10} {'status':>12}"
)
print("-" * 66)
for entry in results_ic:
    r = entry["result"]
    print(
        f"{entry['label']:<18} {r.parameters[phys_offset]:>12.1f} {r.parameters[phys_offset + 1]:>10.3f}"
        f" {r.reduced_chi_squared:>10.4f} {r.convergence_status:>12}"
    )

## 7. Result Visualization

In [ ]:
best_result = result_multi.best
D0_fit = best_result.final_params[phys_offset]
alpha_fit = best_result.final_params[phys_offset + 1]
D_offset_fit = best_result.final_params[phys_offset + 2]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot 1: C2 lag-time curve with model overlay
ax = axes[0]
# Experimental: average over all angles
lag_indices = range(1, n_t - 1)
lag_times = [t[k] for k in lag_indices]
c2_exp_avg = []
for lag_idx in lag_indices:
    n = n_t - lag_idx
    vals = np.array(
        [c2[i_phi, k, k + lag_idx] for i_phi in range(n_phi) for k in range(n)]
    )
    c2_exp_avg.append(np.mean(vals))

# Precompute D(t) on the time grid and its cumulative trapezoid (fitted params)
D_t_fit = D0_fit * t**alpha_fit + D_offset_fit
trap_avg_fit = 0.5 * (D_t_fit[:-1] + D_t_fit[1:])
D_cumsum_fit = np.concatenate([[0.0], np.cumsum(trap_avg_fit)])

# Model (from fitted params)
c2_model_avg = []
for lag_idx in lag_indices:
    # Diffusion integral (cumulative trapezoid)
    D_integral = abs(D_cumsum_fit[lag_idx] - D_cumsum_fit[0]) * dt
    g1_sq = np.exp(-2 * q**2 * D_integral)
    c2_model_avg.append(1.0 + 0.15 * g1_sq)  # using estimated contrast

ax.semilogx(lag_times, c2_exp_avg, "ko", markersize=3, label="Experiment")
ax.semilogx(lag_times, c2_model_avg, "r-", linewidth=2, label="Model fit")
ax.set_xlabel("Lag time (s)")
ax.set_ylabel("⟨C2⟩")
ax.set_title(f"Fit quality: chi²_nu = {best_result.reduced_chi_squared:.3f}")
ax.legend()

# Plot 2: Parameter comparison
ax = axes[1]
param_names_plot = ["D0", "alpha", "D_offset"]
true_vals = [TRUE["D0"], TRUE["alpha"], TRUE["D_offset"]]
fitted_vals = [best_result.final_params[phys_offset + i] for i in range(3)]
# Uncertainties from covariance if available
if best_result.covariance is not None:
    fitted_errs = [
        np.sqrt(best_result.covariance[phys_offset + i, phys_offset + i])
        for i in range(3)
    ]
else:
    fitted_errs = [0.0, 0.0, 0.0]

x = np.arange(3)
width = 0.35
bars = ax.bar(
    x, [abs(tv) for tv in true_vals], width, label="True", alpha=0.6, color="steelblue"
)
ax.bar(
    x + width,
    [abs(fv) for fv in fitted_vals],
    width,
    yerr=[abs(e) for e in fitted_errs],
    capsize=5,
    label="Fitted",
    alpha=0.8,
    color="orange",
)
ax.set_xticks(x + width / 2)
ax.set_xticklabels(param_names_plot)
ax.set_yscale("log")
ax.set_ylabel("|Parameter value|")
ax.set_title("True vs Fitted Parameters (absolute value)")
ax.legend()

plt.tight_layout()
display(fig)
plt.close(fig)

print("\nFitted parameters:")
for name, tv, fv, fe in zip(param_names_plot, true_vals, fitted_vals, fitted_errs):
    rel = abs(fv - tv) / abs(tv) * 100 if tv != 0 else float("nan")
    print(f"  {name}: true={tv:.4g}, fitted={fv:.4g} ± {fe:.4g}  ({rel:.1f}% error)")

## 8. Summary

Key takeaways from static mode analysis:

- **D₀** characterizes particle dynamics; compare to Stokes-Einstein for size
- **alpha** reveals diffusion regime (< 0: sub, 0: normal, > 0: super)
- **Anti-degeneracy** (`per_angle_mode: "auto"`) protects physical parameters
- **Multi-start** provides confidence that the global minimum was found
- **chi²_nu ~ 1** indicates a good fit with well-estimated uncertainties

For rigorous uncertainty quantification, proceed to `04_bayesian_inference.ipynb`.

In [ ]:
import os

os.unlink(config_path)